## LQTMOMENT Tutorial 2: Preparing Wave Data and Instrument Response File

Setting the wave/seismogram data and the instrument response file is a crucial step in calculating moment magnitude using the `lqtmoment` package.


> **ℹ️ INFO ℹ️**
>
> In this current version, `lqtmoment` only accepts `.mseed` format for seismogram data and `RESP` files format for instrument response file. These two formats have become the standard for most organizations nowadays, and maintaining them for processing is considered good practice.


### A. Setting up The Seismogram Directory Structure

Suppose that under the `wave` directory, the seismogram data within this directory must be stored according to the following structure: each seismogram dataset is placed in a folder named with the corresponding earthquake ID.


```
wave/
├── 1001/
│   ├── 19700101T000000_KJ.KJ01_BHE.mseed
│   ├── 19700101T000000_KJ.KJ01_BHN.mseed
│   ├── 19700101T000000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T000000_KJ.KJ02_BHE.mseed
│   ├── 19700101T000000_KJ.KJ02_BHN.mseed
│   ├── 19700101T000000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T000000_KJ.KJ03_BHE.mseed
│   ├── 19700101T000000_KJ.KJ03_BHN.mseed
│   └── 19700101T000000_KJ.KJ03_BHZ.mseed
├── 1002/
│   ├── 19700101T010000_KJ.KJ01_BHE.mseed
│   ├── 19700101T010000_KJ.KJ01_BHN.mseed
│   ├── 19700101T010000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T010000_KJ.KJ02_BHE.mseed
│   ├── 19700101T010000_KJ.KJ02_BHN.mseed
│   ├── 19700101T010000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T010000_KJ.KJ03_BHE.mseed
│   ├── 19700101T010000_KJ.KJ03_BHN.mseed
│   └── 19700101T010000_KJ.KJ03_BHZ.mseed
├── 1003/
│   ├── 19700101T020000_KJ.KJ01_BHE.mseed
│   ├── 19700101T020000_KJ.KJ01_BHN.mseed
│   ├── 19700101T020000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T020000_KJ.KJ02_BHE.mseed
│   ├── 19700101T020000_KJ.KJ02_BHN.mseed
│   ├── 19700101T020000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T020000_KJ.KJ03_BHE.mseed
│   ├── 19700101T020000_KJ.KJ03_BHN.mseed
│   └── 19700101T020000_KJ.KJ03_BHZ.mseed
├── 1004/
│   ├── 19700101T030000_KJ.KJ01_BHE.mseed
│   ├── 19700101T030000_KJ.KJ01_BHN.mseed
│   ├── 19700101T030000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T030000_KJ.KJ02_BHE.mseed
│   ├── 19700101T030000_KJ.KJ02_BHN.mseed
│   ├── 19700101T030000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T030000_KJ.KJ03_BHE.mseed
│   ├── 19700101T030000_KJ.KJ03_BHN.mseed
│   └── 19700101T030000_KJ.KJ03_BHZ.mseed
├── 1005/
│   ├── 19700101T040000_KJ.KJ01_BHE.mseed
│   ├── 19700101T040000_KJ.KJ01_BHN.mseed
│   ├── 19700101T040000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T040000_KJ.KJ02_BHE.mseed
│   ├── 19700101T040000_KJ.KJ02_BHN.mseed
│   ├── 19700101T040000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T040000_KJ.KJ03_BHE.mseed
│   ├── 19700101T040000_KJ.KJ03_BHN.mseed
│   └── 19700101T040000_KJ.KJ03_BHZ.mseed
└── ...

> **⚠️ CAUTION ⚠️**
>
> Ideally, the naming convention for seismogram data in the `lqtmoment` package follows the format `<yyyymmddTHHMMSS>_<network_code>.<station_code>_<channel>.mseed`. However, `lqtmoment` is flexible with naming conventions and primarily relies on the `<station_code>` to identify the corresponding seismogram data. Adhering to the full naming convention for seismogram data is always considered best practice.
> 
>
> The length, or duration, of the wave data for each component of the seismogram is user-defined; make sure it captures the entire earthquake coda. Longer wave data durations will result in heavier loading times, while shorter durations will reduce loading times and vice versa.
>
> Tight trimming and rigorous downsampling can indeed improve loading times, but be cautious of frequency leakage and aliasing. Allow some space for each tip; the tapering will be locally handled by `lqtmoment`.



### B. Preparing Instrument Response File

A digital seismogram typically stores its raw measurements in **count** units. To calculate moment magnitude, which is a physical property, we need to convert these digital units into **true ground displacement in meters (m)**. This is achieved through deconvolution, where the spectrum in digital units is divided by the instrument response function. To perform this accurately, the correct **instrument response file** is required.


**1. Getting Instrument Response File (RESP)**

Typically, your instrument vendor or provider will supply the correct datalogger and sensor response file combined in `RESP` format with every purchase. If these are not included or you get the data from open-source or third-party providers, you can retrieve the correct files through the [Iris Data Web Service](http://service.iris.edu/irisws/nrl/1/) by inputting the appropriate datalogger and sensor specifications.

For example:

* **Datalogger Specs**: REFTEK 130-01; Gain 32; 500 sps
* **Sensor Specs**: CEADASE CPHM-500; Long Period 1; Gain 8; Sensitivity 2250 V/m/s; Sensor Type Ground Vel

The web url query would be like this:

http://service.iris.edu/irisws/nrl/1/combine?instconfig=sensor_CEADASE_CPHM-500_LP1_PG8_SG2250_STgroundVel:datalogger_REFTEK_130-01_PG32_FR500&format=resp&nodata=404 

> **⚠️ CAUTION ⚠️**
>
> After obtaining the `RESP` file, you must still need to configure and align the actual seismogram metadata with the metadata form the `RESP` file you just donwloaded.
>
> Any discrepancy between the RESP metadata and the seismogram metadata will prevent lqtmoment from processing the waveform data, as it cannot parse the poles, zeros, and sensitivity constant from the RESP file.


**2. RESP file format**

A `RESP` file is an ASCII representation of SEED instrument response information. Generally RESP files only contain the subset of SEED headers that represent an instrument response. For more detail please check this out [The RESP format](http://ds.iris.edu/ds/nodes/dmc/data/formats/resp/#:~:text=A%20RESP%20file%20is%20an,metadata%20available%20in%20SEED%20headers.).

A `RESP` file generally contains these informations:

* **Station and Channel Info**: Network, Station, Location, Channel Codes, and Validity dates.
* **Poles and Zeros**: Defines the sensor's frequency response for deconvolution.
* **Sensitivity/Gain**: Scales raw counts to physical units (e.g., counts/m/s)
* **Sampling Rate**: Specifies the digitization frequency (e.g., 20 Hz)
* **Stage Sequence**: Describes the signal path (sensor, digitizer, filters) with units and gains.gains.



**3. Configuring the Directory Structure and Naming Conventions for RESP files**

The naming convention for `RESP` file is following this format.

`RESP.<network_code>.<station_code>.<location>.<channel>.resp`

for example:

`RESP.IU.FURI.00.BHE.resp`

> **⚠️ CAUTION ⚠️**
>
> The naming structure must correspond to the data header in the `RESP` file.

Data header in `RESP` usually contains these informations:

```
# ============= CHANNEL RESPONSE DATA ===============
B050F03        Station:        FURI
B050F16        Network:        IU
B052F03        Location:       00
B052F04        Channel:        BHE
B052F22        Start Date:     1999,111,00:00:00
B052F23        End Date: No Ending Time
```

> **⚠️ CAUTION ⚠️**
>
> Any discrepancy between the `RESP` header metadata and the naming convention will prevent `lqtmoment` from processing the waveform data, as it cannot parse the poles, zeros, and sensitivity constant from the RESP file.



So suppose under `calibration` folder, your directory structure should be looking like this:

```
wave/
├── RESP.IU.FURI.00.BHE.resp
├── RESP.IU.FURI.00.BHN.resp
├── RESP.IU.FURI.00.BHZ.resp
├── RESP.ZQ.AAMI.00.BHE.resp
├── RESP.ZQ.AAMI.00.BHN.resp
├── RESP.ZQ.AAMI.00.BHZ.resp
└── 